# iflearner client Configure and start

## 1. 虚拟环境
可以通过conda、pyenv等虚拟工具创建iflearner的虚拟运行环境，然后激活。

### 1.1. Conda
1.需要安装conda环境。 [安装](https://docs.conda.io/projects/conda/en/stable/user-guide/install/index.html)

2.我们将通过conda创建一个虚拟环境并激活它
```shell
$ conda create -n iflearner python==3.9 ipykernel
$ conda activate iflearner
```

3.将虚拟环境写入jupyter notebook内核
命令：python -m ipykernel install --user --name 虚拟环境名称 --display-name 虚拟环境名称
- 第一个虚拟环境名称表示创建的虚拟环境名称
- 第二个虚拟环境名称表示您希望它出现在 jupyter notebook 的内核选项中

   例如： $ python -m ipykernel install --user --name iflearner --display-name "iflearner"

4.切换内核
可以在jupyterlab的内核菜单栏中选择更改内核按钮，选择我们创建的虚拟环境

## 2.安装iflearner库及相关依赖

In [3]:
!pip install iflearner torch==1.7.1 torchvision==0.8.2  --index-url http://pypi.douban.com/simple --trusted-host pypi.douban.com 


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


Looking in indexes: http://pypi.douban.com/simple/
ERROR: Could not find a version that satisfies the requirement iflearner
ERROR: No matching distribution found for iflearner
Looking in indexes: http://pypi.douban.com/simple/


## 3. 定义客户端并启动

### 3.1. 定义 Pytorch 模型

In [2]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
from torchvision import datasets, transforms

from iflearner.business.homo.argument import parser
from iflearner.business.homo.pytorch_trainer import PyTorchTrainer
from iflearner.business.homo.train_client import Controller


class Model(nn.Module):
    def __init__(self, num_channels, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(num_channels, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

ModuleNotFoundError: No module named 'iflearner.business'

### 3.2. 集成和实现 PytorchTrainer

In [ ]:
class Mnist(PyTorchTrainer):
    def __init__(self, lr=0.15, momentum=0.5) -> None:
        self._lr = lr
        self._device = (
            torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        )
        print(f"device: {self._device}")
        self._model = Model(num_channels=1, num_classes=10).to(self._device)

        super().__init__(self._model)

        self._optimizer = optim.SGD(self._model.parameters(), lr=lr, momentum=momentum)
        self._loss = F.nll_loss

        apply_transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        )
        train_dataset = datasets.MNIST(
            "./data", train=True, download=True, transform=apply_transform
        )
        test_dataset = datasets.MNIST(
            "./data", train=False, download=True, transform=apply_transform
        )
        self._train_data = torch.utils.data.DataLoader(
            train_dataset, batch_size=64, shuffle=True
        )
        self._test_data = torch.utils.data.DataLoader(
            test_dataset, batch_size=64, shuffle=False
        )

    def fit(self, epoch):
        self._model.to(self._device)
        self._model.train()
        print(
            f"Epoch: {epoch}, the size of training dataset: {len(self._train_data.dataset)}, batch size: {len(self._train_data)}"
        )
        for batch_idx, (data, target) in enumerate(self._train_data):
            data, target = data.to(self._device), target.to(self._device)
            self._optimizer.zero_grad()
            output = self._model(data)
            loss = self._loss(output, target)
            loss.backward()
            self._optimizer.step()

    def evaluate(self, epoch):
        self._model.to(self._device)
        self._model.eval()
        test_loss = 0
        correct = 0
        print(f"The size of testing dataset: {len(self._test_data.dataset)}")
        with torch.no_grad():
            for data, target in self._test_data:
                data, target = data.to(self._device), target.to(self._device)
                output = self._model(data)
                test_loss += self._loss(
                    output, target, reduction="sum"
                ).item()  # sum up batch loss
                pred = output.argmax(
                    dim=1, keepdim=True
                )  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(self._test_data.dataset)

        print(
            "Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(
                test_loss,
                correct,
                len(self._test_data.dataset),
                100.0 * correct / len(self._test_data.dataset),
            )
        )

        return {"loss": test_loss, "acc": correct / len(self._test_data.dataset)}

### 3.3. 启动client

In [ ]:
if __name__ == "__main__":
    args = parser.parse_args(args=[])
    print(args)
    args.name = "client02"
    args.epochs = 2
    mnist = Mnist()
    controller = Controller(args, mnist)
    controller.run()